# Model for Nature Conservancy Fisheries Kaggle Competition

#### Dependencies

In [1]:
import fish_data as fd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pandas as pd

#### Helper functions

In [2]:
help(fd)

Help on module fish_data:

NAME
    fish_data

DESCRIPTION
    fish_data module contains the helper functions for the model build of the
    Nature Conservancy Fisheries Kaggle Competition.
    
    Dependencies:
        * numpy as np
        * os
        * scipy.ndimage as ndimage
        * scipy.misc as misc
        * scipy.special as special
        * matplotlib.pyplot as plt
        * tensorflow as tf

FUNCTIONS
    count_nodes(y_in, x_in, conv_depths, conv_strides, pool_strides)
        Calculates the number of total nodes present in the last layer of a
        convolution plus max_pooling architecture.  Calculations assume that
        convolution is 'SAME' padded, and pooling is 'VALID' padded.
    
    decode_image(image_read, size, num_channels=3, mutate=False, brightness_delta=None, contrast_limits=None, hue_delta=None, saturation_limits=None)
        Converts a dequeued image read from filename to a single tensor array,
        with modifications:
            * resized to st

#### Generate a list of filenames

In [3]:
fish_filenames = fd.generate_filenames_list('data/train/', subfolders = True)
print("There are {} filenames in the master set list".format(len(fish_filenames)))
test_filenames = fd.generate_filenames_list('data/test_stg1/', subfolders = False)
print("There are {} filenames in the test set list".format(len(test_filenames)))

There are 3777 filenames in the master set list
There are 1000 filenames in the test set list


#### Generate the labels for the master set list

In [4]:
fish_label_arr = fd.make_labels(fish_filenames, 'train/', '/img')
fish_label_arr.shape
print("One label per row entry: {}".format(all(np.sum(fish_label_arr, 1) == 1) ))

One label per row entry: True


#### Shuffle and split the master set list into training and validation sets

In [5]:
valid_size = 200
files_train, files_val, y_train, y_val = train_test_split(fish_filenames, fish_label_arr, test_size = valid_size)
print("Validation set size: {}".format(y_val.shape[0]))
print("Training set size: {}".format(y_train.shape[0]))

Validation set size: 200
Training set size: 3577


## Graph and Session Runs

#### Graph parameters

In [6]:
num_epochs = 5
#Preprocessing
std_y = 300
std_x = 500

# General
num_channels = 3
num_labels = 8
batch_size = 25
stddev = 0.2

# convolution
kernel_sizes = [5, 3, 3, 3, 3, 3]
conv_depths = [64, 64, 128, 256, 512, 256] # NOTE : first 64 currently not used with dilation2d
conv_strides = [4, 1, 1, 1, 1, 1]

pool_strides = [2, 2, 2, 2]

final_depth = conv_depths[-1]

#dropout
kp = 0.75

# fully connected
fc1_depth = 256
fc2_depth = 64

#regularization
beta = 1e-1 

# Learning rate
init_rate = 5e-2
per_steps = len(files_train)
decay_rate = 0.9

# Report rate
validate_interval = 20


#### Session parameters

In [7]:
version_ID = 'v1.3.3.4'
logs_path = os.getcwd()+'/TB_logs/'+version_ID

In [8]:
%run -i 'GRAPH.py'

In [9]:
%run -i 'SESSION.py'

Initialized!


To view your tensorboard dashboard summary, run the following on the command line:
tensorboard --logdir='/Users/ccthomps/Documents/Python Files/Kaggle Competitions/Nature Conservancy Fisheries/TB_logs/v1.3.3.4'

Batch number: 1
     Training_mean_cross_entropy: 354.5852355957031
     Valid_mean_cross_entropy: 2155.749755859375
Batch number: 5
     Training_mean_cross_entropy: 388.77105712890625
     Valid_mean_cross_entropy: 1834.689697265625
Batch number: 9
     Training_mean_cross_entropy: 115.6520767211914
     Valid_mean_cross_entropy: 137.90646362304688
Batch number: 13
     Training_mean_cross_entropy: 90.01799774169922
     Valid_mean_cross_entropy: 113.59841918945312
Batch number: 17
     Training_mean_cross_entropy: 55.11125183105469
     Valid_mean_cross_entropy: 38.61949157714844
Batch number: 21
     Training_mean_cross_entropy: 26.043590545654297
     Valid_mean_cross_entropy: 11.075347900390625
Batch number: 25
     Training_mean_cross_entropy: 11.639993667

In [10]:
print(test_df)

                            image           ALB  BET           DOL  \
0    data/test_stg1/img_00005.jpg  0.000000e+00  0.0  0.000000e+00   
1    data/test_stg1/img_00007.jpg  0.000000e+00  0.0  1.186109e-35   
2    data/test_stg1/img_00009.jpg  0.000000e+00  0.0  0.000000e+00   
3    data/test_stg1/img_00018.jpg  0.000000e+00  0.0  0.000000e+00   
4    data/test_stg1/img_00027.jpg  0.000000e+00  0.0  0.000000e+00   
5    data/test_stg1/img_00030.jpg  0.000000e+00  0.0  0.000000e+00   
6    data/test_stg1/img_00040.jpg  0.000000e+00  0.0  0.000000e+00   
7    data/test_stg1/img_00046.jpg  0.000000e+00  0.0  0.000000e+00   
8    data/test_stg1/img_00053.jpg  0.000000e+00  0.0  0.000000e+00   
9    data/test_stg1/img_00071.jpg  0.000000e+00  0.0  0.000000e+00   
10   data/test_stg1/img_00075.jpg  0.000000e+00  0.0  0.000000e+00   
11   data/test_stg1/img_00102.jpg  0.000000e+00  0.0  0.000000e+00   
12   data/test_stg1/img_00103.jpg  0.000000e+00  0.0  0.000000e+00   
13   data/test_stg1/